In [ ]:
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroSerializer
from confluent_kafka.serialization import (
    MessageField,
    SerializationContext,
    StringSerializer,
)
from libs.configuration import configure
from libs.kafka import MessageProducer
from libs.models.aircraft import RawAircraft

from aircraft_model import doc8643

env = configure()

In [ ]:
sr_client = SchemaRegistryClient({"url": env.KAFKA_SCHEMA_REGISTRY_URL})
out_schema = sr_client.get_latest_version(f"{env.KAFKA_TOPIC_RAW_AIRCRAFT}-value")
out_schema.schema.schema_str

In [ ]:
string_serializer = StringSerializer("utf_8")
avro_serializer = AvroSerializer(
    schema_registry_client=sr_client,
    schema_str=out_schema.schema.schema_str,
    conf={"auto.register.schemas": False},
)
value_serialization_context = SerializationContext(env.KAFKA_TOPIC_RAW_AIRCRAFT, MessageField.VALUE)
value_serializer = lambda x: avro_serializer(x, value_serialization_context)

p = MessageProducer[RawAircraft](
    topic=env.KAFKA_TOPIC_RAW_AIRCRAFT,
    key_serializer=string_serializer,
    value_serializer=value_serializer,
)

In [ ]:
f = open('../../assets/doc8643_icao.txt')
icao_list = f.read().splitlines()
f.close()

In [ ]:
for x in doc8643.scrape(icao_list):
    p.produce(x)
    # print(x)